In [48]:
df

,label,type_symbol,symmetry_multiplicity,Wyckoff_symbol,fract_x,fract_y,fract_z,B_iso_or_equiv,occupancy
0,Ni1,Ni2+,4,a,0,0,0,.,1.
1,O1,O2-,4,b,0.5,0.5,0.5,.,1.


In [4]:
import numpy as np
import pandas as pd

def get_sym_constraints(cif_file):
    with open(cif_file, 'r') as f:
        lines = f.read().split('\n')
    xyz_index1 = [i for i, l in enumerate(lines) if '_xyz' in l]
    xyz_index2 = [i for i, l in enumerate(lines) if i > xyz_index1[0] and 'loop_' in l]
    xyz_lines = lines[xyz_index1[0]+1:xyz_index2[0]]

    if '_space_group_symop_id' in lines:
        xyz_lines = [xyz[len(f'{i+1}'):] for i, xyz in enumerate(xyz_lines)]

    # clean up the xyz lines remove tabs, commas, and quotes
    for ch in [' ', '\t', "'"]:
        xyz_lines = [line.replace(ch, '') for line in xyz_lines] 
    out = [[],[],[]]
    for xyz in xyz_lines:
        for comp in xyz.split(','):
            comp_ind = [(ind, comp) for ind, char in enumerate(['x', 'y', 'z']) if char in comp]
            for ind, comp in comp_ind:
                out[ind].append(comp)
    xyz_lines = [','.join(comp) for comp in np.array(out).T]
    transforms = [lambda x, y, z, coef=i: eval(f'[{coef}]') for i in xyz_lines]
    return transforms 


def get_fract_cord(cif_file):
    with open(cif_file, 'r') as f:
        lines = f.read().split('\n')
    site_ind = [
        ind for ind, line in enumerate(lines) if line.startswith('_atom_site_')
    ]
    site_slice = slice(site_ind[0], site_ind[-1]+1)
    col_names = [line.split('_atom_site_')[-1] for line in lines[site_slice]]
    l_site = site_ind[-1] + 1
    number_symb_ind = [l_site + i for i, line in enumerate(lines[l_site + 1:])
                       if '#' in line]
    if number_symb_ind:
        data = [line.split() for line in lines[site_ind[-1]+1:number_symb_ind[0]+1]]
    else:
        data = [line.split() for line in lines[site_ind[-1]+1:]]
    df = pd.DataFrame(data, columns=col_names)
    return df

def gen_site_pos(df, transforms):
    frac_label  = {}
    for lab in df.label.unique():  
        row = df.loc[df.label == lab,:]
        x, y, z = [row[xyz].to_numpy()[0] for xyz in ['fract_x', 'fract_y', 'fract_z']]

        x, y, z = [float(i.split('(')[0]) for i in [x, y, z]]
        frac = np.unique([t(x, y, z) for t in transforms], axis=0)
        frac = frac % 1
        frac = np.unique(frac, axis=0)
        frac_label[lab] = frac
    return frac_label

def count_atom(frac_label, df):
    atom_multiplicity = {}
    for label, pos in frac_label.items():
        atom_multiplicity[label] = np.array(pos).shape[0]
    # rescale atom_multiplicity by the occupancy
    for label, occ in zip(df.label, df.occupancy):
        atom_multiplicity[label] *= float(occ)
    return atom_multiplicity
        

cif_file = '/home/cipmin/5_Felix/GitHub_EzFit/cifs/NiO.cif'
df = get_fract_cord(cif_file)
transforms = get_sym_constraints(cif_file)
frac_label = gen_site_pos(df, transforms)
count_atom(frac_label, df)



{'Ni1': 4.0, 'O1': 4.0}

$S_\mathrm{molar_a} = \frac{S_a\cdot(n_a * f1_a^2)^{-1}}{\sum_i S_i\cdot(n_i * f1_i^2)^{-1}}$

In [7]:
denom = 0

denom += 1

denom

1

In [8]:
a += 1

NameError: name 'a' is not defined